# CashFlow

In [1]:
import pandas as pd
import xlwings as xw
from datetime import datetime
from forex_python.converter import get_rate

In [2]:
# set up reporting month and year
month = 1
year = 2022

## ING

In [3]:
# load ING statment
dfING = pd.read_csv('E:\\Gielda\\Porfel\\01JAN22\\ING.csv', 
                    skiprows = 20, encoding='ISO-8859-2', sep = ";")

In [4]:
# select columns
df_ING = dfING[['Data transakcji', 'Dane kontrahenta', 'Tytuł',
       'Nr rachunku', 'Kwota transakcji (waluta rachunku)','Saldo po transakcji']]

# rename columns
df_ING = df_ING.rename(columns = {'Data transakcji':'Date','Dane kontrahenta':'Recipient', 'Tytuł':'Title',
                                  'Nr rachunku':'Account', 'Kwota transakcji (waluta rachunku)':'Value',
                                  'Saldo po transakcji': 'Saldo'}, inplace = False)

In [5]:
# 'Value' to replace string and convert to float
df_ING['Value'] = df_ING['Value'].str.replace(',','.').astype(float)
df_ING['Saldo'] = df_ING['Saldo'].str.replace(',','.').astype(float)

# remoce everything which is NaN
df_ING.dropna(subset = ['Value'], inplace = True)

# change 'Date' to datetime type
df_ING['Date'] = pd.to_datetime(df_ING['Date'])

# add column with source
df_ING['Source'] = 'ING'

## mBank

In [6]:
# Load mBank statment
dfmBank = pd.read_csv('E:\\Gielda\\Porfel\\01JAN22\\mBankAll.csv', 
                      skiprows = 32, encoding='ISO-8859-2', sep = ";", header = None)

In [7]:
# mBank
# select columns
df_mBank = dfmBank[[0,1,4,5]]
df_mBank.columns = ['Date', 'Recipient', 'Value','Saldo']

In [8]:
# replace letters in 'Value' as below
df_mBank['Saldo'] = df_mBank['Saldo'].str[:-4].str.replace(',', '.').str.replace(' ', '')

# change the type to float
df_mBank['Saldo'] = df_mBank['Saldo'].astype(float)

#  change 'Date' to datetime
df_mBank['Date'] = pd.to_datetime(df_mBank['Date'])

# additional column with source name
df_mBank['Source'] = df_mBank.apply(lambda x: 'mBank_' + x['Value'][-3:], axis = 1)

<ipython-input-8-3a6e2d1ab253>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mBank['Saldo'] = df_mBank['Saldo'].str[:-4].str.replace(',', '.').str.replace(' ', '')
<ipython-input-8-3a6e2d1ab253>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mBank['Saldo'] = df_mBank['Saldo'].astype(float)
<ipython-input-8-3a6e2d1ab253>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [9]:
df_mBank['Value'] = df_mBank.apply(lambda x: get_rate(x['Value'][-3:], 'PLN', x['Date'])
                                    * float(x['Value'][:-3].replace(',', '.').replace(' ', '')), axis=1)

<ipython-input-9-d8448ea9f3be>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mBank['Value'] = df_mBank.apply(lambda x: get_rate(x['Value'][-3:], 'PLN', x['Date'])


## Santander

In [10]:
# Load Santander Private
df_Santander_P = pd.read_csv('E:\\Gielda\\Porfel\\01JAN22\\SantanderP.csv',
                           skiprows = 1, header = None)

# additional column with source name
df_Santander_P['Source'] = "Santander_P"

In [12]:
# Load Santander Company
df_Santander_C = pd.read_csv('E:\\Gielda\\Porfel\\01JAN22\\SantanderC1.csv',
                           skiprows = 1, header = None, sep = ",")

# additional column with source name
df_Santander_C['Source'] = "Santander_C"

In [13]:
# concatenate all Santander lists
frames = [df_Santander_P, df_Santander_C]
df_Santander = pd.concat(frames)

In [14]:
# Santader Private
# select columns
df_Santander = df_Santander[[0, 2, 3, 4, 5, 6, 'Source']]

# rename columns
df_Santander = df_Santander.rename(columns = {0:'Date', 2:'Recipient', 3:'Title', 4:'Account', 
                                                5:'Value', 6:'Saldo'}, inplace = False)

In [15]:
# 'Value' to replace string and convert to float
df_Santander['Value'] = df_Santander['Value'].str.replace(',','.').astype(float)
df_Santander['Saldo'] = df_Santander['Saldo'].str.replace(',','.').astype(float)

# change 'Date' to datetime type
df_Santander['Date'] = pd.to_datetime(df_Santander['Date'], format = "%d-%m-%Y")

## Concatenate

In [16]:
# concatenate all the dataframes into one table
frames = [df_ING, df_mBank, df_Santander]
result = pd.concat(frames)

In [17]:
# filter by specific date
result_Current = result.loc[(result['Date'].dt.month == month) & (result['Date'].dt.year == year)]
result_Previous = result.loc[(result['Date'].dt.month == 12) & (result['Date'].dt.year == 2021)]

In [18]:
result_Current = pd.DataFrame(result_Current.groupby(['Source']).first()['Saldo'])
result_Previous = pd.DataFrame(result_Previous.groupby(['Source']).first()['Saldo'])

In [ ]:
result_Current.to_excel('E:\\Gielda\\Porfel\\01JAN22\\Results_31JAN.xlsx')

In [34]:
# export to excel
wb = xw.Book('E:\Gielda\Porfel\Porfel.xlsm')
sheet = wb.sheets['pyScript_Banks']
sheet.range('B2').value = sheet.range('C2:C8').options(ndim=2).value
sheet.range('E1:I10').value = ''
sheet.range('H1').value = result_Current
sheet.range('E1').value = result_Previous